In [1]:
!pip install git+https://github.com/qubvel/efficientnet

  Cloning https://github.com/qubvel/efficientnet to /tmp/pip-req-build-5ygzt4z_
  Running command git clone -q https://github.com/qubvel/efficientnet /tmp/pip-req-build-5ygzt4z_
     |████████████████████████████████| 50 kB 1.5 MB/s 
  Created wheel for efficientnet: filename=efficientnet-1.1.0-py3-none-any.whl size=18397 sha256=4df8ac411006c291e68a7a5259bd928fa96c35b531889603ef4fe8b462440cd1
  Stored in directory: /tmp/pip-ephem-wheel-cache-rdq18npf/wheels/11/69/85/814d64d694c96db0eef17b718042d644a1e54f113920481920
Successfully built efficientnet


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm
import cv2
import random
from scipy.ndimage import rotate
import gc
import datetime
from sklearn.metrics import roc_auc_score

import efficientnet.keras as efn
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Using TensorFlow backend.


In [3]:
dfpaths = []
for i in range(25):
    dfpaths.append("../input/meldataset1202007092115/train_%04d.pkl" % i)
for i in range(25, 50):
    dfpaths.append("../input/meldataset2202007092115/train_%04d.pkl" % i)

In [4]:
DEBUG = 0

In [5]:
if DEBUG == 1:
    print(dfpaths)

In [6]:
#we're gonna time this now
if DEBUG == 1:
    tottimestart = datetime.datetime.now()
    for i in range(50):
        now = datetime.datetime.now()
        tmp = pd.read_pickle(dfpaths[i])
        print("Load time = " + str(datetime.datetime.now() - now))
        print(tmp.shape)
        now = datetime.datetime.now()
        nparr = np.array([i for i in (tmp.loc[:, "image"].to_numpy().tolist())])
        print("Conversion time = " + str(datetime.datetime.now() - now))
        print(nparr.shape)

        now = datetime.datetime.now()
        del tmp
        del nparr
        gc.collect()
        print("gc time = " + str(datetime.datetime.now() - now))
    print("total time = " + str(datetime.datetime.now() - tottimestart))

In [7]:
#del traindata
#del testdata
gc.collect()

20

In [8]:
BATCH_SIZE = 64 #* tpu_strategy.num_replicas_in_sync
STEPS_PER_EPOCH = 760
VAL_STEPS = 190
training_epochs = 30
#model_tests = 1
c2dl1pen = 0#1e-6
c2dl2pen = 01e-5
l1penalty = 0#5e-6
l2penalty = 02e-5

In [9]:
gc.collect()

40

In [10]:
train_aug = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                            rotation_range=60,
                                                            width_shift_range=0.2, 
                                                            height_shift_range=0.2,
                                                            shear_range=0.1,
                                                            horizontal_flip=True,
                                                            vertical_flip=True)
tmp_aug = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                          rotation_range = 30,
                                                          horizontal_flip=True,
                                                          vertical_flip=True)
flip_aug = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                          horizontal_flip=True,
                                                          vertical_flip=True)
val_aug = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [11]:
def makemodel(summarize = False, output_plot = False):
    resnet = efn.EfficientNetB6(include_top=False, weights='imagenet', input_shape = (224, 224 ,3))
    resnet.trainable=False
    flat = tf.keras.layers.Flatten()(resnet.output)
    flat = tf.keras.layers.Dense(256, activation = "relu")(flat)
    flat = tf.keras.layers.Dense(256, activation = "relu")(flat)
    dense = tf.keras.layers.Dense(1, activation = "sigmoid")(flat)
    model = tf.keras.Model(resnet.input, dense)
    opt = tf.keras.optimizers.Adam()
    loss = tf.keras.losses.BinaryCrossentropy(label_smoothing = 0.02)
    auc = tf.keras.metrics.AUC()
    model.compile(loss=loss, metrics=auc, optimizer=opt)
    if(summarize):
        model.summary()
    if(output_plot):
        tf.keras.utils.plot_model(model, to_file="model-visual-%s.png" % datetime.datetime.isoformat(datetime.datetime.now()), show_shapes=True)
    return model

In [12]:
def train_gen_only_flip_ISIC20():
    while True:
        for i in range(0, 40):
            tmpdf = pd.read_pickle(dfpaths[i]).query("source == \"ISIC20\"")
            nparr = np.array([j for j in (tmpdf.loc[:, "image"].to_numpy().tolist())])
            tmpgen = flip_aug.flow(x = nparr,
                                    y = tmpdf.loc[:, "target"].to_numpy(),
                                    batch_size = 64,
                                    shuffle = True)
            for j in range(len(tmpgen)):
                yield tmpgen.__next__()
            del tmpgen
            del nparr
            del tmpdf
            gc.collect()

def train_gen_only_flip():
    while True:
        for i in range(0, 40):
            tmpdf = pd.read_pickle(dfpaths[i])
            nparr = np.array([j for j in (tmpdf.loc[:, "image"].to_numpy().tolist())])
            tmpgen = flip_aug.flow(x = nparr,
                                    y = tmpdf.loc[:, "target"].to_numpy(),
                                    batch_size = 64,
                                    shuffle = True)
            for j in range(len(tmpgen)):
                yield tmpgen.__next__()
            del tmpgen
            del nparr
            del tmpdf
            gc.collect()
def train_gen_aug():
    while True:
        for i in range(0, 40):
            tmpdf = pd.read_pickle(dfpaths[i])
            nparr = np.array([j for j in (tmpdf.loc[:, "image"].to_numpy().tolist())])
            tmpgen = train_aug.flow(x = nparr,
                                    y = tmpdf.loc[:, "target"].to_numpy(),
                                    batch_size = 64,
                                    shuffle = True)
            for j in range(len(tmpgen)):
                yield tmpgen.__next__()
            del tmpgen
            del nparr
            del tmpdf
            gc.collect()
def val_gen():
    while True:
        for i in range(40, 50):
            tmpdf = pd.read_pickle(dfpaths[i])
            nparr = np.array([j for j in (tmpdf.loc[:, "image"].to_numpy().tolist())])
            tmpgen = val_aug.flow(x = nparr,
                                    y = tmpdf.loc[:, "target"].to_numpy(),
                                    batch_size = 64,
                                    shuffle = True)
            for j in range(len(tmpgen)):
                yield tmpgen.__next__()
            del tmpgen
            del nparr
            del tmpdf
            gc.collect()

In [13]:
#for i in range(model_tests):
model = makemodel(summarize = True)
gc.collect()

165527552/165527152 [==============================] - 2s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 112, 112, 56) 1512        input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 112, 112, 56) 224         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 112, 112, 56) 0           stem_bn[0][0]                    
___________________________

3370

In [14]:
callback = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 5, verbose = 1, restore_best_weights = True)

In [15]:
history = model.fit(x = train_gen_only_flip(), epochs = training_epochs, validation_data = val_gen(),
                    steps_per_epoch = STEPS_PER_EPOCH, validation_steps = VAL_STEPS, callbacks = [callback])
model.save("trained_model %s.h5" % datetime.datetime.now().strftime("%Y%m%d_%H%M%S"))
gc.collect()

Epoch 1/30
760/760 [==============================] - 326s 429ms/step - loss: 0.6143 - auc: 0.7533 - val_loss: 0.2802 - val_auc: 0.8589
Epoch 2/30
760/760 [==============================] - 322s 423ms/step - loss: 0.2644 - auc: 0.8296 - val_loss: 0.2525 - val_auc: 0.8698
Epoch 3/30
760/760 [==============================] - 321s 423ms/step - loss: 0.3188 - auc: 0.8397 - val_loss: 0.2412 - val_auc: 0.8730
Epoch 4/30
760/760 [==============================] - 321s 422ms/step - loss: 0.2505 - auc: 0.8551 - val_loss: 0.2469 - val_auc: 0.8781
Epoch 5/30
760/760 [==============================] - 320s 421ms/step - loss: 0.2472 - auc: 0.8626 - val_loss: 0.2431 - val_auc: 0.8762
Epoch 6/30
760/760 [==============================] - 323s 425ms/step - loss: 0.2495 - auc: 0.8600 - val_loss: 0.2474 - val_auc: 0.8751
Epoch 7/30
760/760 [==============================] - 320s 422ms/step - loss: 0.2456 - auc: 0.8644 - val_loss: 0.2372 - val_auc: 0.8874
Epoch 8/30
760/760 [============================

23

In [16]:
gc.collect()

20